In [1]:
from PIL import Image
import glob
import random
import cv2
import numpy as np
import os
import tqdm


In [2]:
base_path = "D:/Documents/13_Projet/dlcv4ldb/02_Donnees"
empty_maps_path = "empty_maps/incarnam"
stickers_path = "stickers/incarnam"

bg_folder_path = f"{base_path}/{empty_maps_path}"
fg_folder_path = f"{base_path}/{stickers_path}"

print(f"{bg_folder_path}")
print(f"{fg_folder_path}")


D:/Documents/13_Projet/dlcv4ldb/02_Donnees/empty_maps/incarnam
D:/Documents/13_Projet/dlcv4ldb/02_Donnees/stickers/incarnam


In [3]:
def get_infos_from_map_name(map_name):
    stem, suf = map_name.split('.')
    stem_split = stem.split("_")
    if len(stem_split) == 5:
        city, area, x, y, level = stem_split
    elif len(stem_split) == 4:
        city, area, x, y = stem_split
        level = None
    else: 
        print(stem_split)
        raise RuntimeError
    return city, area, x, y, level

def get_infos_from_mob_name(mob_name):
    stem, suf = mob_name.split('.')
    stem_split = stem.split("_")
    name, pos, bg = stem_split
    return name, pos, bg

def pick_mob_pos_with_mask(mask_path):
    img = Image.open(mask_path).convert("L")
    img_arr = np.array(img)
    white_pix = list(np.argwhere(img_arr == 255))
    if not white_pix:
        return (None, None)
    else:
        pos_x, pos_y = random.choice(white_pix)
        return (pos_x, pos_y)


In [4]:
classes = [
    "pissenlitMiroitant", 
    "roseVaporeuse", 
    "tofuChimerique", 
    "tournesolNebuleux", 
    "chaferDebutant", 
    "chaferEclaireur", 
    "chaferPiquier", 
    "chafeurFurtif", 
    "chakrobat", 
    "ronronchon", 
    "tigrimas", 
    "grandSplatch", 
    "petitGloot", 
    "plikplok", 
    "aminite", 
    "bouftonOrageux", 
    "bouftonPalichon", 
    "bouftorEthere", 
    "bouftouNuageux", 
    "feuDeJoie", 
    "feuFollet", 
    "feuFurieux", 
    "feuVif", 
    ]


In [5]:
number_of_img = 3
train_size = 0.5
p_mobs = [0.1, 0.2, 0.3, 0.4]
show = True
write = False
color_box = (0, 0, 255)
color_text = (0, 0, 0)
data_rel_path = "../02_Donnees/datasets"
train_img_path = os.path.join(data_rel_path, "dofus", "images", "train")
train_label_path = os.path.join(data_rel_path, "dofus", "labels", "train")
val_img_path = os.path.join(data_rel_path, "dofus", "images", "val")
val_label_path = os.path.join(data_rel_path, "dofus", "labels", "val")
if write and not os.path.exists(train_img_path):
    os.makedirs(train_img_path)
    os.makedirs(train_label_path)
    os.makedirs(val_img_path)
    os.makedirs(val_label_path)
for img_number in tqdm.tqdm(range(number_of_img), desc="Creating dataset"):
    data_name = f"{img_number:08d}"
    bg_area_list = glob.glob(f"{bg_folder_path}/*")
    print(f"{bg_area_list=}")
    bg_area_path = random.choice(bg_area_list)
    bg_list = glob.glob(f"{bg_area_path}/crop/*")
    bg_path = random.choice(bg_list)
    bg_name = bg_path.split("\\")[-1]
    city, area, coord_x, coord_y, level = get_infos_from_map_name(bg_name)
    img = Image.open(bg_path, 'r')
    img_width, img_height = img.size
    mob_list = glob.glob(f"{fg_folder_path}/mobs/{area}/*")
    label = ""
    if mob_list:
        number_of_mobs = np.random.choice(4, 1, p=p_mobs)[0]
        stickers = []
        for j in range(number_of_mobs):
            mob_path = random.choice(mob_list)
            mob_pos_list = glob.glob(f"{mob_path}/final/*")
            mob_pos_path = random.choice(mob_pos_list)
            mob_img = Image.open(mob_pos_path, 'r')
            width, height = mob_img.size
            y, x = pick_mob_pos_with_mask(bg_path.replace("crop", "mask"))
            if x and y:
                mob_name = mob_pos_path.split("\\")[-1]
                name, pos, bg = get_infos_from_mob_name(mob_name)
                class_id = classes.index(name)
                x_center_rel = x / img_width
                y_center_rel = (y - height//2) / img_height
                width_rel = width / img_width
                height_rel = height / img_height
                if x_center_rel < 0:
                    x = width//2
                    x_center_rel = x / img_width
                if y_center_rel < 0:
                    y = height
                    y_center_rel = (y - height//2) / img_height
                x_pasted = x - width//2
                y_pasted = y - height
                img.paste(mob_img, (x_pasted, y_pasted), mask=mob_img)
                label += f"{class_id} {x_center_rel} {y_center_rel} {width_rel} {height_rel}"
                if not j + 1 == number_of_mobs:
                    label += "\n"
                stickers.append({
                    "name": name, "left": x_pasted, "top": y_pasted, "width": width, "height": height
                })
    if write:
        if img_number / number_of_img < train_size:
            img.save(os.path.join(train_img_path, f"{data_name}.png"))
            with open(os.path.join(train_label_path, f"{data_name}.txt"), 'w') as file:
                file.write(label)
        if img_number / number_of_img >= train_size:
            img.save(os.path.join(val_img_path, f"{data_name}.png"))
            with open(os.path.join(val_label_path, f"{data_name}.txt"), 'w') as file:
                file.write(label)
    if show:
        img = np.array(img)[:, :, ::-1].copy()
        for sticker in stickers:
            img = cv2.rectangle(img, (sticker["left"], sticker["top"]), (sticker["left"] + sticker["width"], sticker["top"] + sticker["height"]), color_box, 2)
            (w, h), _ = cv2.getTextSize(sticker["name"], cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            img = cv2.rectangle(img, (sticker["left"], sticker["top"] - 20), (sticker["left"] + w, sticker["top"]), color_box, -1)
            img = cv2.putText(img, sticker["name"], (sticker["left"], sticker["top"] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_text, 1)
        
        cv2.imshow("Display window", img)
        k = cv2.waitKey(0)


Creating dataset:   0%|          | 0/3 [00:00<?, ?it/s]

bg_area_list=[]


IndexError: list index out of range